In [1]:
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import requests
import time
from googletrans import Translator
import os

In [2]:
# # Translate text
# target_language = 'en'
# translator = Translator()
# result = translator.translate(content[1].text, dest=target_language)

# # Print result
# print(result.text)

In [3]:
class crawler:
    '''
    A webscrapper object that is capable of scrapping novels on `https://booktoki287.com` iteratively through selenium webdriver.
    '''
    def __init__(self, initial_chapter_url, headless=True):
        
        self.paragraphs = None
        self.next_ch_url = None
        self.status = False
        # Configure webdriver options
        options = Options()

        # Disable image loading
        options = webdriver.ChromeOptions()
        options.add_experimental_option(
            "prefs", {"profile.managed_default_content_settings.images": 2}
        )
        
        # Do not show the web window
        if headless:
            options.add_argument('--headless')
            
        self.driver = webdriver.Chrome(options=options)
        
        try:
            # Initialize webdriver and get the page
            self.driver.get(initial_chapter_url)
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Get the page source and parse it with BeautifulSoup
            html_content = self.driver.page_source
            self.soup = BeautifulSoup(html_content, 'html.parser')
            # Your scraping logic here
            self.content = self.soup.find('div', {'class', 'view-padding'})
            self.content = self.soup.find_all('p')

            self.__ch_num = 1
            self.get_ch_num()
            
        except:
            self.quit()
        
    def get_ch_num(self):
        '''
        Get the chapter number scrapped from the current html content.
        '''
        self.ch_num = self.soup.find('div', {'class', 'toon-title'}).find_all('span')[0].text.split('/')[0]
        self.ch_num = re.sub("\D", '', self.ch_num)
        
    def get_next_chapter(self):
        '''
        Get the html content for the next chapter.
        '''
        if self.__ch_num == 1:
            self.paragraphs = self.content
            self.__ch_num = None
        else:
            self.next_ch_url = self.soup.find('a', id='goNextBtn')['href']
            if 'next' in self.next_ch_url:
                self.status = True
                return 'The scrapper has run successfully'
            self.driver.get(self.next_ch_url)
            # time.sleep(2)
            self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Get the page source and parse it with BeautifulSoup
            html_content = self.driver.page_source
            self.soup = BeautifulSoup(html_content, 'html.parser')
            # Your scraping logic here
            self.content = self.soup.find('div', {'class', 'view-padding'})
            self.content = self.soup.find_all('p')
            
            self.paragraphs = self.content
            self.get_ch_num()
                
    def __call__(self, save_dir: str=None):

        # change the current working directory
        os.chdir('C:\\Users\\muham\\OneDrive\\Desktop\\Dataset for finetuning MTL-LLM\\')

        # if the save_in path does not exist then create it first
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        while not self.status:
            self.get_next_chapter()
            clear_output()
            # create file_name for the text file
            file_name = os.path.join(save_dir, str(self.ch_num)+'.txt')

            # save the `content` in the text file iteratively.
            with open(file_name, 'w', encoding='utf-8') as f:
                for p in self.paragraphs:
                    f.write(f"{p.text}\n")

            print(f"Done: {self.ch_num}")
            
        
    def quit(self):
        '''
        Quit the selenium webdriver.
        '''
        self.driver.quit()

In [4]:
test = crawler("https://booktoki287.com/novel/4456323", headless=True)

In [9]:
test(save_dir='rdl')

Done: 628


In [ ]:
test.quit()

In [18]:
with open("dataset\\1.txt", 'r', encoding='utf-8') as f:
    a = f.readlines()

In [20]:
with open("translated_chapters\\1.txt", 'r') as f:
    b = f.readlines()

In [10]:
import pandas as pd

In [30]:
org = pd.DataFrame({'raws': a[1:], 'translations': b})

ValueError: All arrays must be of the same length

In [ ]:
org.head()